In [1]:
import numpy as np
import pandas as pd

In [2]:
raw_data = pd.read_excel("/opt/ml/input/data/raw_codishop/view/item/item_tag.xlsx", engine='openpyxl')
all_item = pd.read_excel("/opt/ml/input/data/raw_codishop/view/item/item.xlsx", engine='openpyxl')
item_with_tag_list = list(raw_data["id"].unique())
unique_item_list = list(all_item["id"].unique())
unique_tag_list = list(raw_data["tag"].unique())

In [3]:
cnt = 0
for item in unique_item_list :
    if item not in item_with_tag_list :
        cnt +=1
print(f"{len(unique_item_list)}개의 아이템 중 {cnt}개는 태그가 없습니다.")
print(f"고유한 태그의 수는 {len(unique_tag_list)}개 입니다.")

1251개의 아이템 중 66개는 태그가 없습니다.
고유한 태그의 수는 1881개 입니다.


In [4]:
raw_data.groupby("id")["tag"].apply(list)

id
25868                           [데님, 생지, 슬림, 슬림진, 와이드, 진, 청]
64040                  [가죽, 기본벨트, 데일리, 벨트, 빈티지, 이탈리아가죽, 캐주얼]
80701                                 [그래픽티셔츠, 박스티셔츠, 반팔티셔츠]
86559                                                [SOCKS]
102619       [스케이트보드, 어센틱, 캐리오버, 캔버스, 캔버스운동화, 시그니처아이템, MD추천]
                                 ...                        
2157202                        [비브람, 슬립온, 남성구두, 남자로퍼, 소가죽로퍼]
2201846          [쉐르파, 플리스, 플리스자켓, 빅트레포일자켓, 빅트레포일, 빅트레포일플리스]
2224536             [레이어드룩, 오버핏가디건, 스웨터가디건, 니트가디건, 와플, 와플니트]
2284131    [원턱팬츠, 슬랙스팬츠, 와이드슬랙스, 와이드팬츠, 세미와이드, 슬랙스, 남자와이드팬츠]
2349427            [인스탁스카메라, 즉석카메라, 폴라로이드카메라, 사진인화기, 인스탁스에보]
Name: tag, Length: 1185, dtype: object

태그가 있는 경우, 일반적으로 최소 하나는 아이템의 종류(중분류)를 나타내는 단어이다. 나머지는 핏, 소재, 스타일에 대한 단어이다.
우리가 만약 태그를 검색 키워드로 사용하는 경우, 아이템의 중분류를 태그로 가져와 사용하는 것도 한 가지 방법일 것 같다.
또는, 아예 태그와 중분류를 모두 가져와서 검색 키워드로 사용하는 것도 방법일 것 같다. 

### 태그에서 핏 정보 추출

1. 핏과 관련된 태크 선별

In [5]:
fit_list = ["핏", "와이드", "오버 사이즈", "슬림", "레귤러", "스키니", "루즈", "릴렉스", "퍼팩트", "스트레이트", "캐롯핏", "코어", "벌룬"]
tag_fit_list = list()
for i in range(len(raw_data)) :
    id = raw_data["id"].iloc[i]
    tag = raw_data["tag"].iloc[i]
    for fit in fit_list :
        if fit in tag :
            tag_fit_list.append((id, tag))

In [6]:
item_tag_fit_df = pd.DataFrame(tag_fit_list, columns=["id","fit"])

2. 추출된 핏 관련 태그 형식 통일

- 캐롯핏 : 바지 테이퍼드 핏, 슬림 핏과 유사
- 캐리오버 : 계절에 상관없이 입는 의류를 나타내는 키워드로, 핏과 관련 없음
- 코어핏 : 특정 핏을 나타내지 않음
- 벌룬핏 : 오버핏에 가장 가까움

In [7]:
fit_list = ["와이드", "오버 사이즈", "슬림", "레귤러", "스키니", "루즈", "릴렉스", "퍼팩트", "스트레이트", "캐롯핏", "코어", "벌룬"]

remove_carryover = item_tag_fit_df.loc[item_tag_fit_df["fit"]=="캐리오버"].index
remove_corefit = item_tag_fit_df.loc[item_tag_fit_df["fit"]=="코어핏"].index
item_tag_fit_df.drop(remove_carryover, inplace=True)
item_tag_fit_df.drop(remove_corefit, inplace=True)

In [8]:
for i in range(len(item_tag_fit_df)) :
    tag = item_tag_fit_df["fit"].iloc[i]
    for fit in fit_list :
        if fit in tag :
            item_tag_fit_df["fit"].iloc[i] = fit
    if tag == "캐롯핏" :
        item_tag_fit_df["fit"].iloc[i] = "슬림"
    elif tag == "퍼팩트" or tag == "스탠다드핏" or tag == "스트레이트":
        item_tag_fit_df["fit"].iloc[i] = "레귤러"
    elif tag == "벌룬" :
        item_tag_fit_df["fit"].iloc[i] = "오버 사이즈"
    elif  tag == "릴렉스":
        item_tag_fit_df["fit"].iloc[i] = "루즈"

/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [9]:
item_tag_fit_df.to_excel("/opt/ml/input/data/asset_codishop/view/item/item_fit_from_tag.xlsx", index=False)